# A Guided Tour of Ray Core: Remote Classes

[*Remote Classes*](https://docs.ray.io/en/latest/walkthrough.html#remote-classes-actors)
involve using a `@ray.remote` decorator on a class. 

This implements an [*actor*](https://patterns.eecs.berkeley.edu/?page_id=258) pattern, with properties: *stateful*, *message-passing semantics*

Actors are extremely powerful. They allow you to take a Python class and instantiate it as a microservice that can be queried from other actors and tasks and even other applications.

When you instantiate a remote Actor, a separate worker process is created on the workder node. Other Ray tasks and actors can invoke its methods on that process, mutating its internal state. When the driver exits or when an actor handle goes out of scope, the Python process terminates. Actors can also be terminated manually if needed. The examples code below show all these cases.

<img src="images/actor_and_workders.png" height="60%" width="70%">

---

First, let's start Ray…

In [1]:
import logging
import time
import ray
import random
from random import randint

In [2]:
ray.init(
    ignore_reinit_error=True,
    logging_level=logging.ERROR,
)

{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': '127.0.0.1:6379',
 'object_store_address': '/tmp/ray/session_2022-01-09_16-02-02_386688_8464/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-01-09_16-02-02_386688_8464/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2022-01-09_16-02-02_386688_8464',
 'metrics_export_port': 64373,
 'node_id': 'ee80b50dbd8f028889f106d7eada03329d1a65821f150975c331ce05'}

## 3. Remote Class as a Stateful Actor Pattern

To start, we'll define a class and use the decorator: `@ray.remote`

Let's use Python class and convert that to a remote Actor class and create multiple actor handle instances associated with distinct attributes, such as a name, age, goals scored, etc

In [3]:
@ray.remote
class GoalsScored:
    def __init__ (self, player, age) -> None:
        self._goals = 0
        self._player = player
        self._age = age

    def score (self, goal) -> object:
        self._goals += goal
        return self._goals
       
    def player(self) -> str:
        return self._player
    
    # Any method of the actor can return multiple object refs.
    @ray.method(num_returns=3)
    def stats(self) -> object:
        return self._player, self._age, self._goals

Define three Actors: Rolando, Neymar, Messi

In [4]:
%%time 

ronaldo = GoalsScored.remote("Ronaldo", randint(18, 35))
neymar = GoalsScored.remote("Neymar", randint(18, 35))
messi = GoalsScored.remote("Messi", randint(18, 35))

CPU times: user 39.2 ms, sys: 17.5 ms, total: 56.8 ms
Wall time: 54.6 ms


Update the scores for each player

In [5]:
%%time

ronaldo.score.remote(randint(1, 7))
neymar.score.remote(randint(1, 7))
messi.score.remote(randint(1, 7))

CPU times: user 1.2 ms, sys: 382 µs, total: 1.59 ms
Wall time: 1.51 ms


ObjectRef(32cccd03c567a25446d7826b6c75f9a61670a5fd0100000001000000)

In [6]:
def print_stats(refs):
    for ref in refs:
        print(f"Player: {ray.get(ref.stats.remote())}")

In [7]:
print_stats([ronaldo, neymar, messi])

Player: ['Ronaldo', 19, 1]
Player: ['Neymar', 18, 1]
Player: ['Messi', 28, 3]


You can use list comprehension to iterate over each Actor handle instances, along with `ObjectRef` for their goals scores, maintained by each distinct actor.

Add three goals for for Neymar. This should change Neymar Actor's internal state

In [8]:
[neymar.score.remote(goal) for goal in range(3)]
print_stats([ronaldo, neymar, messi])

Player: ['Ronaldo', 19, 1]
Player: ['Neymar', 18, 4]
Player: ['Messi', 28, 3]


## Tree of Actors Pattern

A common pattern used in Ray libraries ([Ray Tune](https://docs.ray.io/en/latest/tune/index.html) and [Ray Train](https://docs.ray.io/en/latest/train/train.html)) to train models in a parallel or distributed HPO.

In this common pattern, tree of actors, a collection of workers as actors, are managed by a supervisor. For example, you want to train multiple models at the same time, while being able to checkpoint/inspect its state.

<img src="https://docs.ray.io/en/latest/_images/tree-of-actors.svg" width="50%" height="40%">

Let's implement a simple exampel to illustrate this pattern.

In [9]:
STATES = ["RUNNING", "DONE"]

class Model:

    def __init__(self, m:str):
        self._model = m

    def train(self):
        # do some training here
        time.sleep(1)
    
def model_factory(m: str):
    return Model(m)

Create a Worker and Supervisor Actors 

In [10]:
@ray.remote
class Worker(object):
    def __init__(self, m:str):
        self._model = m
        
    def state(self) -> str:
        return random.choice(STATES)
    
    def work(self) -> None:
        model_factory(self._model).train()
         
@ray.remote
class Supervisor:
    def __init__(self):
        # Create three Actor workers, each by its unique name
        self.workers = [Worker.remote(name) for name in ["lr", "cl", "nn"]]
                        
    def work(self):
        [w.work.remote() for w in self.workers]
        
    def terminate(self):
        [ray.kill(w) for w in self.workers]
        
    def state(self):
        return ray.get([w.state.remote() for w in self.workers])

Create a Actor instance for supervisor and launch its workers

In [11]:
sup = Supervisor.remote()

# Launch remote actors as workers
sup.work.remote()

ObjectRef(24eed4584329c19af7b0fd6fbdbe69001b2eee740100000001000000)

### Look at the Ray Dashboard
http://127.0.0.1:8265

In [12]:
# check their status
while True:
    states = ray.get(sup.state.remote())
    print(states)
    result = all('DONE' == e for e in states)
    if result:
        # Note: Actor processes will be terminated automatically when the initial actor handle goes out of scope in Python. 
        # If we create an actor with actor_handle = ActorClass.remote(), then when actor_handle goes out of scope and is destructed, 
        # the actor process will be terminated. Note that this only applies to the original actor handle created for the actor 
        # and not to subsequent actor handles created by passing the actor handle to other tasks.
        
        # kill supervisors all worker manually, only for illustrtation and demo
        sup.terminate.remote()

        # kill the supervisor manually, only for illustration and demo
        ray.kill(sup)
        break

['RUNNING', 'DONE', 'DONE']
['DONE', 'RUNNING', 'DONE']
['DONE', 'RUNNING', 'DONE']
['RUNNING', 'DONE', 'DONE']
['DONE', 'DONE', 'RUNNING']
['RUNNING', 'RUNNING', 'RUNNING']
['RUNNING', 'DONE', 'DONE']
['DONE', 'DONE', 'RUNNING']
['DONE', 'DONE', 'RUNNING']
['RUNNING', 'DONE', 'DONE']
['RUNNING', 'RUNNING', 'RUNNING']
['RUNNING', 'RUNNING', 'RUNNING']
['RUNNING', 'DONE', 'DONE']
['RUNNING', 'DONE', 'DONE']
['DONE', 'DONE', 'DONE']


### Passing Actor handles to Ray Tasks

You can pass actor handle instances to remote Ray tasks, which can change its 
state. The MessageActor keeps or clears messages, depending on the its method
invoked.

In [13]:
@ray.remote
class MessageActor(object):
    def __init__(self):
        self.messages = []
    
    def add_message(self, message):
        self.messages.append(message)
    
    def get_and_clear_messages(self):
        messages = self.messages
        self.messages = []
        return messages

Define a remote function which loops around and pushes messages to the actor, having access to a handle instance as an argument. That is, we are sendit a message handle ref as an argument to it.

In [14]:
@ray.remote
def worker(message_actor, j):
    for i in range(10):
        time.sleep(1)
        message_actor.add_message.remote(
            f"Message {i} from worker {j}.")


Create a message actor.

In [15]:
message_actor = MessageActor.remote()

Start 3 tasks that push messages to the actor.

In [16]:
[worker.remote(message_actor, j) for j in range(3)]

[ObjectRef(d5a75db31f99bd73ffffffffffffffffffffffff0100000001000000),
 ObjectRef(d68fec326c8433c9ffffffffffffffffffffffff0100000001000000),
 ObjectRef(8e088f779f48acd6ffffffffffffffffffffffff0100000001000000)]

Periodically get the messages and print them.

In [17]:
for _ in range(10):
    new_messages = ray.get(message_actor.get_and_clear_messages.remote())
    print("New messages\n:", new_messages)
    time.sleep(1)

New messages
: []
New messages
: ['Message 0 from worker 1.', 'Message 0 from worker 0.']
New messages
: ['Message 0 from worker 2.', 'Message 1 from worker 1.', 'Message 1 from worker 0.']
New messages
: ['Message 1 from worker 2.', 'Message 2 from worker 1.', 'Message 2 from worker 0.', 'Message 2 from worker 2.']
New messages
: ['Message 3 from worker 1.', 'Message 3 from worker 0.', 'Message 3 from worker 2.']
New messages
: ['Message 4 from worker 1.', 'Message 4 from worker 0.', 'Message 4 from worker 2.']
New messages
: ['Message 5 from worker 0.', 'Message 5 from worker 2.', 'Message 5 from worker 1.']
New messages
: ['Message 6 from worker 1.', 'Message 6 from worker 0.', 'Message 6 from worker 2.']
New messages
: ['Message 7 from worker 1.', 'Message 7 from worker 2.', 'Message 7 from worker 0.']
New messages
: ['Message 8 from worker 1.', 'Message 8 from worker 0.', 'Message 8 from worker 2.']


Finally, shutdown Ray

In [18]:
ray.shutdown()

---
## References

 * [Using and Programming with Actors](https://docs.ray.io/en/latest/actors.html)
 * [Advanced Patterns and Anti-Patterns in Ray](https://docs.ray.io/en/latest/ray-design-patterns/index.htmlhttps://docs.ray.io/en/latest/ray-design-patterns/index.html)